<a href="https://colab.research.google.com/github/hfelizzola/Timetabling-Unisalle/blob/main/Time_Tabling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Time Tabling Model

In [1]:
# Instalar paquete ortools
!pip install ortools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 16.3 MB 33.9 MB/s 
     |████████████████████████████████| 408 kB 52.0 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.21.9 which is incompatible.
tensorflow-metadata 1.11.0 requires protobuf<4,>=3.13, but you have protobuf 4.21.9 which is incompatible.
tensorboard 2.9.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.21.9 which is incompatible.


In [2]:
from ortools.linear_solver import pywraplp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [3]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [4]:
os.chdir('/gdrive/MyDrive/Colab Notebooks/Time Tabling')

In [69]:
!ls

 consulta.xlsx	  'Tablas para GAMS VF1.xlsx'
 input_data.xlsx   Time_Tabling.ipynb


In [70]:
# Crear el modelo
#solver = pywraplp.Solver.CreateSolver('SCIP')
solver = pywraplp.Solver.CreateSolver('GLOP')

## Conjuntos

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-0pky{border-color:inherit;text-align:left;vertical-align:top}
</style>
<table class="tg">
<thead>
  <tr>
    <th class="tg-0pky">   <br>Subíndice   </th>
    <th class="tg-0pky">   <br>Descripción   </th>
    <th class="tg-0pky">   <br>Dominio   </th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-0pky">   <br>i   </td>
    <td class="tg-0pky">   <br>Franjas Horarias   </td>
    <td class="tg-0pky">   <br>i = 1,2,…,11   </td>
  </tr>
  <tr>
    <td class="tg-0pky">   <br>j   </td>
    <td class="tg-0pky">   <br>Asignatura   </td>
    <td class="tg-0pky">   <br>j = 1,2,…,90   </td>
  </tr>
  <tr>
    <td class="tg-0pky">   <br>k   </td>
    <td class="tg-0pky">   <br>Docentes   </td>
    <td class="tg-0pky">   <br>k = 1,2,…,29   </td>
  </tr>
  <tr>
    <td class="tg-0pky">   <br>n   </td>
    <td class="tg-0pky">   <br>Días hábiles   </td>
    <td class="tg-0pky">   <br>n = 1,2,…,6   </td>
  </tr>
  <tr>
    <td class="tg-0pky">   <br>m   </td>
    <td class="tg-0pky">   <br>Semestre al que   pertenecer una asignatura   </td>
    <td class="tg-0pky">   <br>m = 1,2,…,10   </td>
  </tr>
  <tr>
    <td class="tg-0pky">   <br>t   </td>
    <td class="tg-0pky">   <br>Bloques de franjas   horarias de dos horas   </td>
    <td class="tg-0pky">   <br>t = 1,2,…,5   </td>
  </tr>
  <tr>
    <td class="tg-0pky">   <br>q   </td>
    <td class="tg-0pky">   <br>Bloques de franjas   horarias de tres horas   </td>
    <td class="tg-0pky">   <br>q = 1,2,3   </td>
  </tr>
</tbody>
</table>

In [71]:
franja = ['F'+str(i) for i in np.arange(1,12,1)] # i
asignatura = ['M'+str(i) for i in np.arange(1,91,1)] # j
docente = ['P'+str(i) for i in np.arange(1,30,1)] # k
dia = ['D'+str(i) for i in np.arange(1,7,1)] # n
semestre = ['S'+str(i) for i in np.arange(1,12,1)] # m
bloques_dos_horas = ['T'+str(i) for i in np.arange(1,6,1)] # t
bloques_tres_horas = ['Q'+str(i) for i in np.arange(1,4,1)] # Q

## Parametros

|      Parámetro     |      Descripción                                                                                                                                 |      subíndice     |
|--------------------|--------------------------------------------------------------------------------------------------------------------------------------------------|--------------------|
|     INTH           |     Número de franjas horarias que se deben dictar de forma   presencial, semanalmente, de la asignatura j.                                      |     j              |
|     PMAX           |     Número de franjas horarias que debe dictar como máximo, el   docente k, a la semana                                                          |     k              |
|     PMIN           |     Número de franjas horarias que debe dictar como mínimo, el   docente k, a la semana                                                          |     k              |
|     DOSF           |     Parámetro binario donde 1 significa que la asignatura j se dicta en bloques de franjas   horarias de dos horas y 0 de lo contrario           |     j              |
|     TRESF          |     Parámetro binario donde 1 significa que la asignatura j se dicta en bloques de franjas   horarias de tres horas y donde 0 de lo contrario    |     j              |
|     MS             |     Matriz binaria donde 1 significa que la asignatura j pertenece al semestre m y 0 de lo contrario                                             |     j,m            |
|     MP             |     Matriz binaria donde 1 significa que la asignatura j puede ser asignada al docente k y donde 0 de lo contrario                               |     j,k            |
|     FP             |     Matriz binaria donde 1 significa que en el día n en la franja i está disponible el docente k   y 0 de lo contrario                           |     n,i,k          |
|     CFM            |     Matriz con ponderación de costo estimado de insatisfacción   de asignar en el día n en la   franja i la asignatura j                         |     n,i,j          |
|     MSF            |     Máximo de salones disponibles por franja horaria para   asignar las diferentes asignaturas                                                   |                    |
|     F              |     Costo total estimado de insatisfacción generado por el   horario asignado                                                                    |                    |

In [72]:
INTH = pd.read_excel('input_data.xlsx', sheet_name='INTH')
INTH = {i:j for i,j in zip(INTH['asignatura'],INTH['INTH'])}
INTH

{'M1': 2.0,
 'M2': 4.0,
 'M3': 4.0,
 'M4': 6.0,
 'M5': 4.0,
 'M6': 4.0,
 'M7': 6.0,
 'M8': 6.0,
 'M9': 4.0,
 'M10': 4.0,
 'M11': 2.0,
 'M12': 2.0,
 'M13': 4.0,
 'M14': 4.0,
 'M15': 4.0,
 'M16': 4.0,
 'M17': 4.0,
 'M18': 4.0,
 'M19': 4.0,
 'M20': 4.0,
 'M21': 6.0,
 'M22': 6.0,
 'M23': 6.0,
 'M24': 6.0,
 'M25': 6.0,
 'M26': 6.0,
 'M27': 6.0,
 'M28': 4.0,
 'M29': 4.0,
 'M30': 4.0,
 'M31': 4.0,
 'M32': 4.0,
 'M33': 4.0,
 'M34': 4.0,
 'M35': 4.0,
 'M36': 3.0,
 'M37': 3.0,
 'M38': 3.0,
 'M39': 3.0,
 'M40': 3.0,
 'M41': 3.0,
 'M42': 3.0,
 'M43': 3.0,
 'M44': 3.0,
 'M45': 3.0,
 'M46': 3.0,
 'M47': 3.0,
 'M48': 3.0,
 'M49': 3.0,
 'M50': 3.0,
 'M51': 3.0,
 'M52': 3.0,
 'M53': 3.0,
 'M54': 3.0,
 'M55': 3.0,
 'M56': 3.0,
 'M57': 3.0,
 'M58': 3.0,
 'M59': 3.0,
 'M60': 3.0,
 'M61': 3.0,
 'M62': 3.0,
 'M63': 3.0,
 'M64': 3.0,
 'M65': 3.0,
 'M66': 3.0,
 'M67': 3.0,
 'M68': 3.0,
 'M69': 3.0,
 'M70': 3.0,
 'M71': 3.0,
 'M72': 3.0,
 'M73': 3.0,
 'M74': 3.0,
 'M75': 3.0,
 'M76': 3.0,
 'M77': 3.0,
 'M78': 

In [73]:
# PMAX
PMAX = pd.read_excel('input_data.xlsx', sheet_name='PMAX')
PMAX = {i:j for i,j in zip(PMAX['docente'],PMAX['PMAX'])}
PMAX

{'P1': 20.0,
 'P2': 20.0,
 'P3': 20.0,
 'P4': 20.0,
 'P5': 20.0,
 'P6': 20.0,
 'P7': 20.0,
 'P8': 20.0,
 'P9': 18.0,
 'P10': 18.0,
 'P11': 18.0,
 'P12': 20.0,
 'P13': 18.0,
 'P14': 20.0,
 'P15': 18.0,
 'P16': 16.0,
 'P17': 20.0,
 'P18': 18.0,
 'P19': 20.0,
 'P20': 20.0,
 'P21': 20.0,
 'P22': 15.0,
 'P23': 20.0,
 'P24': 20.0,
 'P25': 20.0,
 'P26': 20.0,
 'P27': 20.0,
 'P28': 10.0,
 'P29': 12.0}

In [74]:
PMIN = pd.read_excel('input_data.xlsx', sheet_name='PMIN')
PMIN = {i:j for i,j in zip(PMIN['docente'],PMIN['PMIN'])}
PMIN

{'P1': 0.0,
 'P2': 0.0,
 'P3': 0.0,
 'P4': 0.0,
 'P5': 0.0,
 'P6': 0.0,
 'P7': 0.0,
 'P8': 0.0,
 'P9': 0.0,
 'P10': 0.0,
 'P11': 0.0,
 'P12': 0.0,
 'P13': 0.0,
 'P14': 0.0,
 'P15': 0.0,
 'P16': 0.0,
 'P17': 0.0,
 'P18': 0.0,
 'P19': 0.0,
 'P20': 0.0,
 'P21': 0.0,
 'P22': 0.0,
 'P23': 0.0,
 'P24': 0.0,
 'P25': 0.0,
 'P26': 0.0,
 'P27': 0.0,
 'P28': 0.0,
 'P29': 0.0}

In [75]:
DOSF = pd.read_excel('input_data.xlsx', sheet_name='DOSF')
DOSF = {i:j for i,j in zip(DOSF['asignatura'],DOSF['DOSF'])}
DOSF

{'M1': 1.0,
 'M2': 1.0,
 'M3': 1.0,
 'M4': 1.0,
 'M5': 1.0,
 'M6': 1.0,
 'M7': 1.0,
 'M8': 1.0,
 'M9': 1.0,
 'M10': 1.0,
 'M11': 1.0,
 'M12': 1.0,
 'M13': 1.0,
 'M14': 1.0,
 'M15': 1.0,
 'M16': 1.0,
 'M17': 1.0,
 'M18': 1.0,
 'M19': 1.0,
 'M20': 1.0,
 'M21': 1.0,
 'M22': 1.0,
 'M23': 1.0,
 'M24': 1.0,
 'M25': 1.0,
 'M26': 1.0,
 'M27': 1.0,
 'M28': 1.0,
 'M29': 1.0,
 'M30': 1.0,
 'M31': 1.0,
 'M32': 1.0,
 'M33': 1.0,
 'M34': 1.0,
 'M35': 1.0,
 'M36': 0.0,
 'M37': 0.0,
 'M38': 0.0,
 'M39': 0.0,
 'M40': 0.0,
 'M41': 0.0,
 'M42': 0.0,
 'M43': 0.0,
 'M44': 0.0,
 'M45': 0.0,
 'M46': 0.0,
 'M47': 0.0,
 'M48': 0.0,
 'M49': 0.0,
 'M50': 0.0,
 'M51': 0.0,
 'M52': 0.0,
 'M53': 0.0,
 'M54': 0.0,
 'M55': 0.0,
 'M56': 0.0,
 'M57': 0.0,
 'M58': 0.0,
 'M59': 0.0,
 'M60': 0.0,
 'M61': 0.0,
 'M62': 0.0,
 'M63': 0.0,
 'M64': 0.0,
 'M65': 0.0,
 'M66': 0.0,
 'M67': 0.0,
 'M68': 0.0,
 'M69': 0.0,
 'M70': 0.0,
 'M71': 0.0,
 'M72': 0.0,
 'M73': 0.0,
 'M74': 0.0,
 'M75': 0.0,
 'M76': 0.0,
 'M77': 0.0,
 'M78': 

In [76]:
TRESF = pd.read_excel('input_data.xlsx', sheet_name='TRESF')
TRESF = {i:j for i,j in zip(TRESF['asignatura'],TRESF['TRESF'])}
TRESF

{'M1': 0.0,
 'M2': 0.0,
 'M3': 0.0,
 'M4': 0.0,
 'M5': 0.0,
 'M6': 0.0,
 'M7': 0.0,
 'M8': 0.0,
 'M9': 0.0,
 'M10': 0.0,
 'M11': 0.0,
 'M12': 0.0,
 'M13': 0.0,
 'M14': 0.0,
 'M15': 0.0,
 'M16': 0.0,
 'M17': 0.0,
 'M18': 0.0,
 'M19': 0.0,
 'M20': 0.0,
 'M21': 0.0,
 'M22': 0.0,
 'M23': 0.0,
 'M24': 0.0,
 'M25': 0.0,
 'M26': 0.0,
 'M27': 0.0,
 'M28': 0.0,
 'M29': 0.0,
 'M30': 0.0,
 'M31': 0.0,
 'M32': 0.0,
 'M33': 0.0,
 'M34': 0.0,
 'M35': 0.0,
 'M36': 1.0,
 'M37': 1.0,
 'M38': 1.0,
 'M39': 1.0,
 'M40': 1.0,
 'M41': 1.0,
 'M42': 1.0,
 'M43': 1.0,
 'M44': 1.0,
 'M45': 1.0,
 'M46': 1.0,
 'M47': 1.0,
 'M48': 1.0,
 'M49': 1.0,
 'M50': 1.0,
 'M51': 1.0,
 'M52': 1.0,
 'M53': 1.0,
 'M54': 1.0,
 'M55': 1.0,
 'M56': 1.0,
 'M57': 1.0,
 'M58': 1.0,
 'M59': 1.0,
 'M60': 1.0,
 'M61': 1.0,
 'M62': 1.0,
 'M63': 1.0,
 'M64': 1.0,
 'M65': 1.0,
 'M66': 1.0,
 'M67': 1.0,
 'M68': 1.0,
 'M69': 1.0,
 'M70': 1.0,
 'M71': 1.0,
 'M72': 1.0,
 'M73': 1.0,
 'M74': 1.0,
 'M75': 1.0,
 'M76': 1.0,
 'M77': 1.0,
 'M78': 

In [77]:
MS = pd.read_excel('input_data.xlsx', sheet_name='MS')
MS = MS.melt(id_vars='asignatura', value_vars=semestre, var_name='semestre', value_name='MS')
MS = {(j,m):res for j,m,res in zip(MS['asignatura'],MS['semestre'],MS['MS'])}
MS

{('M1', 'S1'): 0.0,
 ('M2', 'S1'): 0.0,
 ('M3', 'S1'): 0.0,
 ('M4', 'S1'): 0.0,
 ('M5', 'S1'): 0.0,
 ('M6', 'S1'): 0.0,
 ('M7', 'S1'): 0.0,
 ('M8', 'S1'): 0.0,
 ('M9', 'S1'): 0.0,
 ('M10', 'S1'): 0.0,
 ('M11', 'S1'): 0.0,
 ('M12', 'S1'): 0.0,
 ('M13', 'S1'): 0.0,
 ('M14', 'S1'): 0.0,
 ('M15', 'S1'): 0.0,
 ('M16', 'S1'): 0.0,
 ('M17', 'S1'): 0.0,
 ('M18', 'S1'): 0.0,
 ('M19', 'S1'): 0.0,
 ('M20', 'S1'): 0.0,
 ('M21', 'S1'): 0.0,
 ('M22', 'S1'): 0.0,
 ('M23', 'S1'): 0.0,
 ('M24', 'S1'): 0.0,
 ('M25', 'S1'): 0.0,
 ('M26', 'S1'): 0.0,
 ('M27', 'S1'): 0.0,
 ('M28', 'S1'): 0.0,
 ('M29', 'S1'): 0.0,
 ('M30', 'S1'): 0.0,
 ('M31', 'S1'): 0.0,
 ('M32', 'S1'): 0.0,
 ('M33', 'S1'): 0.0,
 ('M34', 'S1'): 0.0,
 ('M35', 'S1'): 0.0,
 ('M36', 'S1'): 0.0,
 ('M37', 'S1'): 0.0,
 ('M38', 'S1'): 0.0,
 ('M39', 'S1'): 0.0,
 ('M40', 'S1'): 0.0,
 ('M41', 'S1'): 0.0,
 ('M42', 'S1'): 0.0,
 ('M43', 'S1'): 1.0,
 ('M44', 'S1'): 0.0,
 ('M45', 'S1'): 0.0,
 ('M46', 'S1'): 0.0,
 ('M47', 'S1'): 0.0,
 ('M48', 'S1'): 0.0,
 

In [78]:
MP = pd.read_excel('input_data.xlsx', sheet_name='MP')
MP = MP.melt(id_vars='asignatura', value_vars=docente, var_name='docente', value_name='MP')
MP = {(j,k):res for j,k,res in zip(MP['asignatura'],MP['docente'],MP['MP'])}
MP

{('M1', 'P1'): 1.0,
 ('M2', 'P1'): 0.0,
 ('M3', 'P1'): 0.0,
 ('M4', 'P1'): 0.0,
 ('M5', 'P1'): 0.0,
 ('M6', 'P1'): 0.0,
 ('M7', 'P1'): 0.0,
 ('M8', 'P1'): 0.0,
 ('M9', 'P1'): 0.0,
 ('M10', 'P1'): 0.0,
 ('M11', 'P1'): 0.0,
 ('M12', 'P1'): 0.0,
 ('M13', 'P1'): 0.0,
 ('M14', 'P1'): 0.0,
 ('M15', 'P1'): 0.0,
 ('M16', 'P1'): 0.0,
 ('M17', 'P1'): 0.0,
 ('M18', 'P1'): 0.0,
 ('M19', 'P1'): 0.0,
 ('M20', 'P1'): 0.0,
 ('M21', 'P1'): 0.0,
 ('M22', 'P1'): 0.0,
 ('M23', 'P1'): 0.0,
 ('M24', 'P1'): 0.0,
 ('M25', 'P1'): 0.0,
 ('M26', 'P1'): 0.0,
 ('M27', 'P1'): 0.0,
 ('M28', 'P1'): 0.0,
 ('M29', 'P1'): 0.0,
 ('M30', 'P1'): 0.0,
 ('M31', 'P1'): 0.0,
 ('M32', 'P1'): 0.0,
 ('M33', 'P1'): 0.0,
 ('M34', 'P1'): 0.0,
 ('M35', 'P1'): 0.0,
 ('M36', 'P1'): 0.0,
 ('M37', 'P1'): 0.0,
 ('M38', 'P1'): 0.0,
 ('M39', 'P1'): 0.0,
 ('M40', 'P1'): 0.0,
 ('M41', 'P1'): 0.0,
 ('M42', 'P1'): 0.0,
 ('M43', 'P1'): 0.0,
 ('M44', 'P1'): 0.0,
 ('M45', 'P1'): 0.0,
 ('M46', 'P1'): 0.0,
 ('M47', 'P1'): 0.0,
 ('M48', 'P1'): 0.0,
 

In [79]:
franja

['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11']

In [80]:
FP = pd.read_excel('input_data.xlsx', sheet_name='FP')
FP = FP.melt(id_vars=['dia','franja'], value_vars=docente, var_name='docente', value_name='FP')
FP = {(n,i,k):value for n,i,k,value in zip(FP['dia'],FP['franja'],FP['docente'],FP['FP'])}
FP

{('D1', 'F1', 'P1'): 1.0,
 ('D1', 'F2', 'P1'): 1.0,
 ('D1', 'F3', 'P1'): 1.0,
 ('D1', 'F4', 'P1'): 1.0,
 ('D1', 'F5', 'P1'): 1.0,
 ('D2', 'F1', 'P1'): 1.0,
 ('D2', 'F2', 'P1'): 1.0,
 ('D2', 'F3', 'P1'): 1.0,
 ('D2', 'F4', 'P1'): 1.0,
 ('D2', 'F5', 'P1'): 1.0,
 ('D3', 'F1', 'P1'): 1.0,
 ('D3', 'F2', 'P1'): 1.0,
 ('D3', 'F3', 'P1'): 1.0,
 ('D3', 'F4', 'P1'): 1.0,
 ('D3', 'F5', 'P1'): 1.0,
 ('D4', 'F1', 'P1'): 1.0,
 ('D4', 'F2', 'P1'): 1.0,
 ('D4', 'F3', 'P1'): 1.0,
 ('D4', 'F4', 'P1'): 1.0,
 ('D4', 'F5', 'P1'): 1.0,
 ('D5', 'F1', 'P1'): 1.0,
 ('D5', 'F2', 'P1'): 1.0,
 ('D5', 'F3', 'P1'): 1.0,
 ('D5', 'F4', 'P1'): 1.0,
 ('D5', 'F5', 'P1'): 1.0,
 ('D6', 'F1', 'P1'): 1.0,
 ('D6', 'F2', 'P1'): 1.0,
 ('D6', 'F3', 'P1'): 1.0,
 ('D6', 'F4', 'P1'): 0.0,
 ('D6', 'F5', 'P1'): 0.0,
 ('D1', 'F1', 'P2'): 1.0,
 ('D1', 'F2', 'P2'): 1.0,
 ('D1', 'F3', 'P2'): 1.0,
 ('D1', 'F4', 'P2'): 1.0,
 ('D1', 'F5', 'P2'): 1.0,
 ('D2', 'F1', 'P2'): 1.0,
 ('D2', 'F2', 'P2'): 1.0,
 ('D2', 'F3', 'P2'): 1.0,
 ('D2', 'F4'

In [81]:
FP_mod = {}
for n in dia:
  for k in docente:
    for i in franja:
      if i == 'F1':
        FP_mod[(n,'F1',k)] = FP[(n,i,k)]
        FP_mod[(n,'F2',k)] = FP[(n,i,k)]
      if i == 'F2':
        FP_mod[(n,'F3',k)] = FP[(n,i,k)]
        FP_mod[(n,'F4',k)] = FP[(n,i,k)]
      if i == 'F3':
        FP_mod[(n,'F5',k)] = FP[(n,i,k)]
        FP_mod[(n,'F6',k)] = FP[(n,i,k)]
        FP_mod[(n,'F7',k)] = 0
      if i == 'F4':
        FP_mod[(n,'F8',k)] = FP[(n,i,k)]
        FP_mod[(n,'F9',k)] = FP[(n,i,k)]
      if i == 'F5':
        FP_mod[(n,'F10',k)] = FP[(n,i,k)]
        FP_mod[(n,'F11',k)] = FP[(n,i,k)]

In [82]:
# CFM	Matriz con ponderación de costo estimado de insatisfacción de asignar en el día n en la franja i la asignatura j
CFM = pd.read_excel('input_data.xlsx', sheet_name='CFM')
CFM.fillna(10, inplace=True)
CFM = CFM.melt(id_vars=['dia','franja'], value_vars=asignatura, var_name='asignatura', value_name='CFM')
CFM = {(n,i,j):value for n,i,j,value in zip(CFM['dia'],CFM['franja'],CFM['asignatura'],CFM['CFM'])}
CFM

{('D1', 'F1', 'M1'): 8.0,
 ('D1', 'F2', 'M1'): 5.0,
 ('D1', 'F3', 'M1'): 6.0,
 ('D1', 'F4', 'M1'): 4.0,
 ('D1', 'F5', 'M1'): 5.0,
 ('D2', 'F1', 'M1'): 8.0,
 ('D2', 'F2', 'M1'): 5.0,
 ('D2', 'F3', 'M1'): 6.0,
 ('D2', 'F4', 'M1'): 4.0,
 ('D2', 'F5', 'M1'): 5.0,
 ('D3', 'F1', 'M1'): 8.0,
 ('D3', 'F2', 'M1'): 5.0,
 ('D3', 'F3', 'M1'): 6.0,
 ('D3', 'F4', 'M1'): 4.0,
 ('D3', 'F5', 'M1'): 5.0,
 ('D4', 'F1', 'M1'): 8.0,
 ('D4', 'F2', 'M1'): 5.0,
 ('D4', 'F3', 'M1'): 6.0,
 ('D4', 'F4', 'M1'): 4.0,
 ('D4', 'F5', 'M1'): 5.0,
 ('D5', 'F1', 'M1'): 8.0,
 ('D5', 'F2', 'M1'): 5.0,
 ('D5', 'F3', 'M1'): 6.0,
 ('D5', 'F4', 'M1'): 4.0,
 ('D5', 'F5', 'M1'): 5.0,
 ('D6', 'F1', 'M1'): 8.0,
 ('D6', 'F2', 'M1'): 5.0,
 ('D6', 'F3', 'M1'): 6.0,
 ('D6', 'F4', 'M1'): 4.0,
 ('D6', 'F5', 'M1'): 5.0,
 ('D1', 'F1', 'M2'): 8.0,
 ('D1', 'F2', 'M2'): 1.0,
 ('D1', 'F3', 'M2'): 4.0,
 ('D1', 'F4', 'M2'): 1.0,
 ('D1', 'F5', 'M2'): 8.0,
 ('D2', 'F1', 'M2'): 8.0,
 ('D2', 'F2', 'M2'): 1.0,
 ('D2', 'F3', 'M2'): 4.0,
 ('D2', 'F4'

In [83]:
CFM_mod = {}
for n in dia:
  for j in asignatura:
    for i in franja:
      if i == 'F1':
        CFM_mod[(n,'F1',j)] = CFM[(n,i,j)]
        CFM_mod[(n,'F2',j)] = CFM[(n,i,j)]
      if i == 'F2':
        CFM_mod[(n,'F3',j)] = CFM[(n,i,j)]
        CFM_mod[(n,'F4',j)] = CFM[(n,i,j)]
      if i == 'F3':
        CFM_mod[(n,'F5',j)] = CFM[(n,i,j)]
        CFM_mod[(n,'F6',j)] = CFM[(n,i,j)]
        CFM_mod[(n,'F7',j)] = 10
      if i == 'F4':
        CFM_mod[(n,'F8',j)] = CFM[(n,i,j)]
        CFM_mod[(n,'F9',j)] = CFM[(n,i,j)]
      if i == 'F5':
        CFM_mod[(n,'F10',j)] = CFM[(n,i,j)]
        CFM_mod[(n,'F11',j)] = CFM[(n,i,j)]
CFM_mod

{('D1', 'F1', 'M1'): 8.0,
 ('D1', 'F2', 'M1'): 8.0,
 ('D1', 'F3', 'M1'): 5.0,
 ('D1', 'F4', 'M1'): 5.0,
 ('D1', 'F5', 'M1'): 6.0,
 ('D1', 'F6', 'M1'): 6.0,
 ('D1', 'F7', 'M1'): 10,
 ('D1', 'F8', 'M1'): 4.0,
 ('D1', 'F9', 'M1'): 4.0,
 ('D1', 'F10', 'M1'): 5.0,
 ('D1', 'F11', 'M1'): 5.0,
 ('D1', 'F1', 'M2'): 8.0,
 ('D1', 'F2', 'M2'): 8.0,
 ('D1', 'F3', 'M2'): 1.0,
 ('D1', 'F4', 'M2'): 1.0,
 ('D1', 'F5', 'M2'): 4.0,
 ('D1', 'F6', 'M2'): 4.0,
 ('D1', 'F7', 'M2'): 10,
 ('D1', 'F8', 'M2'): 1.0,
 ('D1', 'F9', 'M2'): 1.0,
 ('D1', 'F10', 'M2'): 8.0,
 ('D1', 'F11', 'M2'): 8.0,
 ('D1', 'F1', 'M3'): 8.0,
 ('D1', 'F2', 'M3'): 8.0,
 ('D1', 'F3', 'M3'): 1.0,
 ('D1', 'F4', 'M3'): 1.0,
 ('D1', 'F5', 'M3'): 4.0,
 ('D1', 'F6', 'M3'): 4.0,
 ('D1', 'F7', 'M3'): 10,
 ('D1', 'F8', 'M3'): 1.0,
 ('D1', 'F9', 'M3'): 1.0,
 ('D1', 'F10', 'M3'): 8.0,
 ('D1', 'F11', 'M3'): 8.0,
 ('D1', 'F1', 'M4'): 8.0,
 ('D1', 'F2', 'M4'): 8.0,
 ('D1', 'F3', 'M4'): 5.0,
 ('D1', 'F4', 'M4'): 5.0,
 ('D1', 'F5', 'M4'): 6.0,
 ('D1', '

In [84]:
MSF = 9

In [85]:
# Número de franjas horarias de la asignatura j
#TRESF = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
#temp = pd.DataFrame({'asignatura':asignatura, 'TRESF':TRESF})
#CFM.to_excel('consulta.xlsx', index=False)

## Variables de Decisión

Se utiliza un total de cuatro variables binarias. 
- La primera consiste en las variables básicas $X_{(i,j,k,n)}$, la cual toma el valor de 1 si en la franja horaria i se asigna la asignatura j al docente k en el día n. 
- El segundo conjunto comprende las variables auxiliares, la variable $Y_{(j,k)}$, la cual toma el valor de 1 si el docente j va a dictar la asignatura k. 
- La variable $H_{(j,k,n,t)}$, H toma el valor de 1 si se activa para asignatura j dictada por el docente k en el día n. 
- La variable $G_{(j,k,n,q)}$, toma el valor de 1 si se activa para asignatura j dictada por el docente k en el día n. 

Las últimas dos variables se hacen necesarias ya que la programación de asignaturas de 4 horas se hace en 2 bloques de 2 horas mientras, las asignaturas de 3 horas se realizan en un solo bloque.

In [86]:
# Creación de la variables X(i,j,k,n)
X = {}
for i in franja:
  for j in asignatura:
    for k in docente:
      for n in dia:
        X[i, j, k, n] = solver.IntVar(0, 1, '')

In [87]:
# Creación de la variables Y(j,k)
Y = {}
for j in asignatura:
  for k in docente:
      Y[j, k] = solver.IntVar(0, 1, '')

In [88]:
# Creación de la variables H(j,k,n,t)
H = {}
for j in asignatura:
  for k in docente:
    for n in dia:
      for t in bloques_dos_horas:
        H[j, k, n, t] = solver.IntVar(0, 1, '')

In [89]:
# Creación de la variables H(j,k,n,t)
G = {}
for j in asignatura:
  for k in docente:
    for n in dia:
      for q in bloques_tres_horas:
        G[j, k, n, q] = solver.IntVar(0, 1, '')

## Función Objetivo

El modelo de programación entera tiene como objetivo la minimización del costo:

$$
\text{Min} \: F = \sum_{i \in I}\sum_{j \in J}\sum_{k \in K}\sum_{n \in N}CFM_{nij} \times X_{ijkn}
$$

In [90]:

solver.Minimize(solver.Sum(CFM_mod[n,i,j]*X[i,j,k,n] for i in franja for j in asignatura for k in docente for n in dia))


## Restricciones

### Ecuación 2

La ecuación (2) asigna un curso a un docente en una franja horaria y día.

$$
\sum_{k \in K} X_{ijkn} \leq 1 \:\:\: \forall i, \forall j, \forall n  
$$


In [91]:
for i in franja:
  for j in asignatura:
    for n in dia:
      solver.Add(solver.Sum([X[i, j, k, n] for k in docente]) <= 1)

### Ecuación 3

La ecuación (3) determina que cada asignatura solo sea asignada como máximo una vez en un respectivo día y franja horaria dictada por un docente.

$$
\sum_{j \in J} X_{ijkn} \leq 1 \:\:\: \forall i, \forall k, \forall n  
$$

In [92]:
for i in franja:
  for k in docente:
    for n in dia:
      solver.Add(solver.Sum([X[i, j, k, n] for j in asignatura]) <= 1)

### Ecuación 4

La ecuación (4) a (6) se conoce como restricciones de completitud, establece que toda asignatura sea programada según su intensidad horaria semanal

$$
\sum_{i \in I} \sum_{k \in K} \sum_{n \in N} X_{ijkn} = INTH_{j} \:\:\: \forall j
$$


In [93]:
for j in asignatura:
  solver.Add(solver.Sum([X[i, j, k, n] for i in franja for k in docente for n in dia]) <= INTH[j])

### Ecuación 5

La ecuación (5) se encarga que cada asignatura solo sea dictada por un único docente, esto significa que, si una asignatura se da en tres momentos diferentes de la semana, en los tres momentos la clase sea dictada por el mismo docente y no que en alguno de los momentos asigne un docente diferente para la asignatura.

$$
\sum_{i \in I} \sum_{n \in N} X_{ijkn} = INTH_{j} \times Y_{jk} \:\:\: \forall j, \forall k
$$

In [94]:
for j in asignatura:
  for k in docente:
    solver.Add(solver.Sum([X[i, j, k, n] for i in franja for n in dia]) <= INTH[j]*Y[j,k])

### Ecuación 6

La restricción indicada en la ecuación (6) establece que un docente solo sea asignado para dictar una asignatura en un respectivo día y franja horaria, si cuenta con la disponibilidad requerida

$$
X_{ijkn} \leq FP_{nik} \:\:\: \forall i, \forall j, \forall k, \forall n
$$


In [95]:
for i in franja:
  for j in asignatura:
    for k in docente:
      for n in dia:
        solver.Add(X[i, j, k, n] <= FP_mod[(n,i,k)])

### Ecuación 7

Las ecuaciones (7) y (8) se encargan de asignar los docentes con las habilidades requeridas y limita el número de cursos que puede dictar el docente durante un día, estableciendo como máximo 6 franjas horarias. 

$$
X_{ijkn} \leq MP_{jk} \:\:\: \forall i, \forall j, \forall k, \forall n
$$

In [96]:
for i in franja:
  for j in asignatura:
    for k in docente:
      for n in dia:
        solver.Add(X[i, j, k, n] <= MP[(j,k)])

### Ecuación 8

$$
\sum_{i \in I} \sum_{j \in J} X_{i,j,k,n} \leq 6 \:\:\: \forall k, \forall n 
$$

In [97]:
  for k in docente:
    for n in dia:
      solver.Add(solver.Sum([X[i, j, k, n] for i in franja for j in asignatura]) <= 6)

### Ecuación 9

La expresión de la ecuación (9) se encarga que las asignaturas definidas como pertenecientes a un mismo semestre se programen en diferentes momentos, es decir en diferente día o franja. 

$$
\sum_{j \in MS_{jm}} \sum_{k \in K} MS_{jm} \times X_{ijkn}; \:\:\: \forall i, \forall n, \forall m
$$

In [98]:
for i in franja:
  for n in dia:
    for m in semestre:
      restriccion9 = []
      for j in asignatura:
        for k in docente:
          restriccion9.append(MS[(j,m)]*X[i,j,k,n])
      solver.Add(solver.Sum(restriccion9)<=1)

### Ecuación 10 y 11

La restricción (10) y (11) establece los limites inferiores y superiores de carga horario para cada docente.

$$
\sum_{i \in I}\sum_{j \in J}\sum_{n \in N}X_{ijkn} \leq PMAX_{k}; \:\:\: \forall k
$$

$$
\sum_{i \in I}\sum_{j \in J}\sum_{n \in N}X_{ijkn} \geq PMIN_{k}; \:\:\: \forall k
$$

In [99]:
for k in docente:
    solver.Add(solver.Sum(X[i, j, k, n] for i in franja for j in asignatura for n in dia) <= PMAX[k])

In [100]:
for k in docente:
  solver.Add(solver.Sum(X[i, j, k, n] for i in franja for j in asignatura for n in dia) >= PMIN[k])

### Ecuaciones 12 13 14 15 16

La ecuación (12) determina que las asignaturas que se dictan en bloque de dos horas se puedan asignar en cualquier día hábil en el espacio académico conformado por las franjas horarias de 7:00 a 9:00 (F1 y F2), es decir para establecer el primer bloque de dos horas (T1). La ecuación (13) se encarga que las asignaturas que se dictan en bloque de dos horas se puedan asignar en cualquier día hábil en el espacio académico conformado por las franjas horarias de 9:00 a 11:00 (F3 y F4), es decir para establecer el segundo bloque de dos horas (T2). 
La expresión indicada en la ecuación (14) determina que las asignaturas que se dictan en bloque de dos horas se puedan asignar en cualquier día hábil en el espacio académico conformado por las franjas horarias de 11:00 a 13:00 (F5 y F6), es decir para establecer el tercer bloque de dos horas (T3). La restricción indicada en la ecuación (15) establece que las asignaturas que se dictan en bloque de dos horas se puedan asignar en cualquier día hábil en el espacio académico conformado por las franjas horarias de 14:00 a 16:00 (F8 y F9), es decir para establecer el cuarto bloque de dos horas (T4). 

$$
X_{F1jkn} + X_{F2jkn} = 2 \times H_{jknT1}; \:\:\: \forall i, \forall k \forall n, \forall j \in DOSF_{j}\\
X_{F3jkn} + X_{F4jkn} = 2 \times H_{jknT2}; \:\:\: \forall i, \forall k \forall n, \forall j \in DOSF_{j}\\
X_{F5jkn} + X_{F6jkn} = 2 \times H_{jknT3}; \:\:\: \forall i, \forall k \forall n, \forall j \in DOSF_{j}\\
X_{F8jkn} + X_{F9jkn} = 2 \times H_{jknT4}; \:\:\: \forall i, \forall k \forall n, \forall j \in DOSF_{j}\\
X_{F10jkn} + X_{F11jkn} = 2 \times H_{jknT5}; \:\:\: \forall i, \forall k \forall n, \forall j \in DOSF_{j}
$$

In [101]:
for i in franja:
  for j in asignatura:
    if DOSF[j] == 1:
        for k in docente:
          for n in dia:
            solver.Add(X['F1', j, k, n] + X['F2', j, k, n] == 2*H[j,k,n,'T1'])
            solver.Add(X['F3', j, k, n] + X['F4', j, k, n] == 2*H[j,k,n,'T2'])
            solver.Add(X['F5', j, k, n] + X['F6', j, k, n] == 2*H[j,k,n,'T3'])
            solver.Add(X['F8', j, k, n] + X['F9', j, k, n] == 2*H[j,k,n,'T4'])
            solver.Add(X['F10', j, k, n] + X['F11', j, k, n] == 2*H[j,k,n,'T5'])
    

### Ecuacion 17

$$
X_{F7jkn} = 0; \:\:\: \forall i, \forall k \forall n, \forall j \in DOSF_{j}
$$

In [102]:
for i in franja:
  if i == 'F7':
    for j in asignatura:
      if (DOSF[j] == 1):
          for k in docente:
            for n in dia:
              solver.Add(X[i,j,k,n] == 1)

### Ecuación 18

La ecuación (18) se encarga que una asignatura que se dicte en bloque de dos horas solo se pueda asignar máximo en uno de los bloques definidos para estas asignaturas, es decir que una de estas asignaturas no se pueda asignar en dos bloques el mismo día. 

$$
\sum_{t \in T}\sum_{k \in K} H_{jknt} \leq 1; \:\:\: \forall j \forall n
$$


In [103]:
for j in asignatura:
  for n in dia:
    solver.Add(solver.Sum(H[j,k,n,t] for t in bloques_dos_horas for k in docente) <= 1)

### Ecuaciones 19 20 21

La restricción indicada en la ecuación (19) establece que las asignaturas que se dictan en bloque de tres horas se puedan asignar en cualquier día hábil en el espacio académico conformado por las franjas horarias de 7:00 a 10:00 (F1, F2 y F3), es decir para establecer el primer bloque de tres horas (Q1). La ecuación (20) establece que las asignaturas que se dictan en bloque de tres horas se puedan asignar en cualquier día hábil en el espacio académico conformado por las franjas horarias de 10:00 a 13:00 (F4, F5 y F6), es decir para establecer el segundo bloque de tres horas (Q2). 
En la restricción indicada en la ecuación (21) se establece que las asignaturas que se dictan en bloque de tres horas se puedan asignar en cualquier día hábil en el espacio académico conformado por las franjas horarias de 13:00 a 16:00 (F7, F8 y F9), es decir para establecer el tres bloque de tres horas (Q3).  

$$
X_{F1jkn} + X_{F2jkn} + X_{F3jkn} = 3 \times G_{jknQ1}; \:\:\: \forall i, \forall k \forall n, \forall j \in TRESF_{j}\\
X_{F4jkn} + X_{F5jkn} + X_{F6jkn} = 3 \times G_{jknQ2}; \:\:\: \forall i, \forall k \forall n, \forall j \in TRESF_{j}\\
X_{F7jkn} + X_{F8jkn} + X_{F9jkn} = 3 \times G_{jknQ3}; \:\:\: \forall i, \forall k \forall n, \forall j \in TRESF_{j}\\
$$


In [104]:
for i in franja:
  for j in asignatura:
    if TRESF[j] == 1:
        for k in docente:
          for n in dia:
            solver.Add(X['F1', j, k, n] + X['F2', j, k, n] + X['F3', j, k, n] == 3*G[j,k,n,'Q1'])
            solver.Add(X['F4', j, k, n] + X['F5', j, k, n] + X['F6', j, k, n] == 3*G[j,k,n,'Q2'])
            solver.Add(X['F7', j, k, n] + X['F8', j, k, n] + X['F9', j, k, n] == 3*G[j,k,n,'Q3'])

### Ecuaciones 22 23

La expresión de la ecuación (22) se encarga que en la franja horaria de 16:00 a 17:00 (F10) no se asigne ninguna de las asignaturas que se dictan en bloque de tres horas.  La restricción indicada en la ecuación (23) se determina que en la franja horaria de 17:00 a 18:00 (F11) no se asigne ninguna de las asignaturas que se dictan en bloque de tres horas. 

$$
X_{F10jkn} = 0; \:\:\: \forall i, \forall k, \forall n, \forall j \in TRESF_j\\
X_{F11jkn} = 0; \:\:\: \forall i, \forall k, \forall n, \forall j \in TRESF_j
$$

In [105]:
for i in franja:
  if (i == 'F10') | (i == 'F11'):
    for j in asignatura:
      if TRESF[j] == 1:
        for k in docente:
          for n in dia:
            solver.Add(X[i,j,k,n] == 0)

### Ecuación 24

La expresión de la ecuación (24) se encarga de que se respete el límite de salones permitidos por franja horaria para el Programa de acuerdo con lo estipulado por la Universidad. 

$$
\sum_{j \in J}\sum_{k \in K} X_{ijkn} \leq MSF; \:\:\: \forall i, \forall n
$$


In [106]:
for i in franja:
  for n in dia:
    solver.Add(solver.Sum(X[i,j,k,n] for j in asignatura for k in docente) <= MSF)

## Solucion

In [107]:
status = solver.Solve()

In [108]:
status == pywraplp.Solver.OPTIMAL

False

In [109]:
solver.Objective().Value()

0.0

In [110]:
X

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7f202428ff60> >